---
# The Vehicle Routing Problems


1. Una buena descripción y muy concisa de problemas de ruteo de vehiculos la encuentras [aqui](https://gbksoft.com/blog/how-to-solve-the-vehicle-routing-problem/)

1. Un modelo cor restricciones de eliminacion de subtours basadas en flujo [aqui](https://www.youtube.com/watch?v=-DjyO0DK9Ys). Flota homogenea, las variables no hacen distincion entre un vehiculo y otro.

1. Complemento de excel con costo [aqui](https://www.zagetdoo.com/vehicle-routing)

1. Platica sobre caso real en Sidney [aqui](https://www.youtube.com/watch?v=BZA_UaX8rs8)

1. Implementando el modelo para el CVRP con gurobi + Python [aqui](https://www.youtube.com/watch?v=7_-Xuq2xKdc)

1. Usando VRP Spreadsheet Solver de excel para resolver algunas variantes del VRP [aquí](https://www.youtube.com/watch?v=enCBp2lBn64), [aquí](https://www.youtube.com/watch?v=APCNU46kxg4), o [aquí](https://www.youtube.com/watch?v=v1v3zusJPuo)   

Algunas comparativas entre diferentes sofware comercial para la optimización de rutas:

1. [Comparison 1](https://www.g2.com/categories/route-planning) 
2. [Comparison 2](https://www.softwareadvice.com/fleet-management/route-planning-comparison/) 
3. [Comparison 3](https://www.spaceotechnologies.com/best-route-optimization-software/)


Manejo adicional de mapas

1. Importar un listado de ubicaciones en Google Maps para visualizar [aqui](https://www.youtube.com/watch?v=y_SkOVshgGY)
2. Agregar rutas de hubicaciones en Google Maps [aqui](https://www.youtube.com/watch?v=_aVVo4mHA_A)

---
## Modelo A 
Usado en [este](https://www.youtube.com/watch?v=7_-Xuq2xKdc) video

Considere que en el modelo del video $x_{ii}$ **no son usadas** y que eso puede viene especificado en el modelo. Además aqui $n$ representa el numero de clientes, no el numero total de puntos.<br>

#### Parámetros

<ul>
<li> $n$: The number of clients  clients) </li>
<li> $N$: set of clients with $N=\{1,2,...,n\}$
<li> $V$: set of vertices (or nodes), with $V=\{0\} \cup N$
<li> $A$: set of arcs, with $A=\{(i,j) \in V^2: i\neq j\}$    
<li> $c_{ij}$: the cost of travel over arc $(i,j)\in A$ </li>
<li> $q_i$: The amount that has to be delivered to costumer $i\in N$
<li> $Q$: Capacity of each truck   
</ul>

#### Variables

$x_{ij}$= 1 if a truck goes from node $i$ to node $j$<br>
$u_{j}$= number of units in a truck when leaves node $j$

#### Model

$$\mbox{Minize} \sum_{i,j \in A}  d_{ij}x_{ij}$$
Subjecto to:
\begin{align}
& \sum_{j \in N, j\neq i} x_{ij} =1 & i \in N\\
& \sum_{i \in N, j\neq i} x_{ij} =1 & j \in N\\
& \mbox{if } x_{ij} =1 \implies u_i + q_j=u_j &  i,j \in A: j \neq 0, i \neq 0\\
& q_i \leq u_{i} \leq Q & i \in N\\
& x_{ij} \in \{0,1\} & i,j \in A
\end{align}

Cabe destacar que la restricción de eliminacion de subtours puede hacer su funcion ya que se decida usarla para especificar entregas o recogidas de mercancia, y que las $u_j$ representen la cantidad de producto que el camión lleva al momento de llegar al cliente $j$ o al momento de salir del cliente $j$; o que las $u_j$ representen el espacio disponible en el camión al momento de llegar al cliente $j$ o al momento de salir del cliente $j$. 

_**La restricción usada puede variar ligeramente, pero debe ser congruente con lo que se desea modelar.**_

Para este problema, las $q_i$ dice que son las cantidades que deben ser **entregadas** al cliente $i$ por lo que si las $u_i$ son la carga en el camión estas deben ir disminuyendo cada que se pasa por un cliente, ahora restaría revisar si son la carga al llegar o al salir del cliente j. Por la restriccion $u_i \leq Q$ se deduce que $u_i$ deberia representar la carga al llegar al cliente i.<br>

Entonces si $x_{ij}=1$ la carga al llegar al cliente j deberia ser igual a la carga que el camión tenia al llegar al cliente i **menos** la cantidad entregada en el cliente $i$... $\mbox{if } x_{ij} =1 \implies u_i - q_i=u_j$.

Es interesante que en el video se use la restricción $\mbox{if } x_{ij} =1 \implies u_i + q_j=u_j$, pero que en la notebook se haya escrito... $\mbox{if } x_{ij} =1 \implies u_i + q_i=u_j$. Ambas ecuaciones en lugar de entrega, representar recogida de producto. La primera de ellas representaria que la carga del vehículo al salir del cliente $j$ es igual a la carga del vehículo al salir del cliente $i$ más la cantidad recogida en el cliente $j$. La segunda representaria que la carga al llegar al cliente $j$ es igual a la carga al llegar al cliente $i$ más la cantidad recogida en el cliente $i$.
 

In [68]:
#importar librerias
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [71]:
def create_instance_vrp(n, m, Q, seed):
    N = range(1,n)
    V = range(0,n)
    random.seed(seed)
    points= [( random.randint(-100,100), random.randint(-100,100)) for i in N]
    points = [(0,0)] + points
    
    dist = {(i,j): math.sqrt((points[i][0] - points[j][0])**2 + (points[i][1]-points[j][1])**2)     for i in V for j in V if i!=j}
    
    dem = [random.randint(1,10) for i in V]
    dem[0] = 0
    
    return n, m, points, dist, dem, Q

In [73]:
n, k, points, dist, dem, Q = create_instance_vrp(10, 2, 20, None)

In [50]:
m = gp.Model("ModelA_CVRP")
N = range(1,n)
V = range(0,n)

#Definir variables

x = m.addVars( dist.keys(), obj=dist.values(), vtype= GRB.BINARY, name="x")
u = m.addVars( N, vtype=GRB.INTEGER, name="u")


---
## Modelo B
Usado en [este](https://www.youtube.com/watch?v=-DjyO0DK9Ys) video

Considere que en el modelo del video $x_{ii}$ son usadas y que eso puede generar errores al resolver el problema. A menos que se modifique el modelo o que se modifique la instancia

#### Parámetros

<ul>
<li> n: The number or points (1-depot, 2,...,n- clients) </li>
<li> $d_{ij}$: distance from point i to point j </li>
<li> $D_j$: Demand of cient $i$
<li> $C$: Capacity of each truck   
</ul>

#### Variables

$x_{ij}$= 1 if a truck goes from node $i$ to node $j$<br>
$f_{ij}$= number of units in a truck going from node $i$ to node $j$

#### Model

$$\mbox{Minize} \sum_{i=1}^n \sum_{j=1}^n d_{ij}x_{ij}$$
Subjecto to:
\begin{align}
& \sum_{j=1}^n x_{ij} =1 & \forall i=2,\dots,n\\
& \sum_{i=1}^n x_{ij} =1 & \forall j=2,\dots,n\\
& \sum_{j=1}^n f_{ij} - \sum_{j=1}^n f_{ij} = D_i & \forall i=2,\dots,n\\
& 0 \leq f_{ij} \leq Cx_{ij} & \forall i,j=1,\dots,n\\
& x_{ij} \in \{0,1\} & \forall i,j= 1,\dots,n
\end{align}

In [68]:
#importar librerias
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [71]:
def create_instance_vrp(n, m, Q, seed):
    N = range(1,n)
    V = range(0,n)
    random.seed(seed)
    points= [( random.randint(-100,100), random.randint(-100,100)) for i in N]
    points = [(0,0)] + points
    
    dist = {(i,j): math.sqrt((points[i][0] - points[j][0])**2 + (points[i][1]-points[j][1])**2)     for i in V for j in V}
    
    dem = [random.randint(1,10) for i in V]
    dem[0] = 0
    
    return n, m, points, dist, dem, Q

In [73]:
n, k, points, dist, dem, Q = create_instance_vrp(10, 2, 20, None)

In [50]:
m = gp.Model("ModelB_CVRP")
N = range(1,n)
V = range(0,n)

#Definir variables

x = m.addVars( dist.keys(), obj=dist.values(), vtype= GRB.BINARY, name="x")
f = m.addVars( dist.keys(), vtype=GRB.CONTINUOUS, name="f")
